#  The Battle of Neighborhoods

# 1. Introduction

### This project is to take a glimpse at different neighborhoods of Bengaluru which is a capital city of an Indian state Karnataka. <br> We will find out different venues which are trending in each neighborrhood and create clusters. <br>People who want to open their business  can look at the result of this project and make decision where can they get more profit.

## 2. Data

### I have organized  data sets of zipcode , neighborhoods with latitude and longtude for Bengaluru city from multiple online sources.


## 3. Methodology

### We will extract the venues of all the neighborhoods for Bengaluru City and then cluster them based on different categories like : Coffee shops, park, etc. <br> For getting all the venue we will use Foursquare API. <br> For clustering we will use K-means algorithm of ML to cluster the neighborhoods <br> For visualizing geographic details of the city I used folium ibrary. 


### Lets start by importing the data from the csv file I have organized from different sources.

In [1]:
# The code was removed by Watson Studio for sharing.

,Zipcode,Name,Latitude,Longitude
0,560071,Domlur,12.9608,77.6361
1,560038,Indiranagar,12.9699,77.6499
2,560003,Malleswaram,13.0031,77.5643
3,560001,Shivajinagar,12.9857,77.6057
4,560103,Bellandur,12.9034,77.6784


## Now that the data is ready , we will start with importing and installing required libraries. 

In [4]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

## Lets see the map with markers of Bengaluru City

In [108]:
address_blr = 'Bengaluru, KA'
address_hy = 'Hyderabad, Telangana'

geolocator = Nominatim(user_agent="ny_explorer")

location_blr = geolocator.geocode(address_blr)
latitude_blr = location_blr.latitude
longitude_blr = location_blr.longitude
print('The geograpical coordinate of Bengaluru are {}, {}.'.format(latitude_blr, longitude_blr))

The geograpical coordinate of Bengaluru are 12.9791198, 77.5912997.


## We have got the coordinates of Bengaluru

## Bengaluru Map

In [6]:
bengaluru_map = folium.Map(location=[latitude_blr, longitude_blr], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_blr['Latitude'], df_blr['Longitude'], df_blr['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(bengaluru_map)  
    
bengaluru_map

## Now lets explore the neighboorhoods of the city

### We will use Foursquare to get the details of the neighborhoods

In [8]:
CLIENT_ID = 'W20CPWKZ0CWMSF2J0UZ3UVO5HZMATNQ34QTHUCOH0H1KEWQE' # your Foursquare ID
CLIENT_SECRET = 'FIPL5BPIDBRAFMOCCLQKAOELCMYY2Q4RKRORR5OFETZ4VZHI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W20CPWKZ0CWMSF2J0UZ3UVO5HZMATNQ34QTHUCOH0H1KEWQE
CLIENT_SECRET:FIPL5BPIDBRAFMOCCLQKAOELCMYY2Q4RKRORR5OFETZ4VZHI


## Now we will create a function to retrieve all the venues for all the neighborhoods of Bengaluru

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Now we will call the above function for Bengaluru and create a new dataframe called bengaluru_venues

In [20]:
bengaluru_venues = getNearbyVenues(names=df_blr['Name'],
                                   latitudes=df_blr['Latitude'],
                                   longitudes=df_blr['Longitude']
                                  )


Domlur
Indiranagar
Malleswaram
Shivajinagar
Bellandur
Hoodi
Marathalli
Whitefield
Kammanahalli
Hebbal
Yeshwanthpur
RT Nagar
Yelahanka
Bommanahalli
Bommasandra
BTM Layout
Electronic City
Koramangala
Jaya Nagar


In [21]:
bengaluru_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Domlur,12.9608,77.6361,League of Extraordinary Gamers,12.967099,77.636919,Gaming Cafe
1,Domlur,12.9608,77.6361,Lavonne,12.963909,77.638579,Café
2,Domlur,12.9608,77.6361,Smoke House Deli,12.965584,77.641498,Deli / Bodega
3,Domlur,12.9608,77.6361,Barbeque Nation,12.962684,77.641599,BBQ Joint
4,Domlur,12.9608,77.6361,Murphy's,12.953659,77.639397,Irish Pub
5,Domlur,12.9608,77.6361,California Burrito,12.967528,77.636722,Burrito Place
6,Domlur,12.9608,77.6361,Starbucks,12.965649,77.641718,Coffee Shop
7,Domlur,12.9608,77.6361,Puma Social Club,12.967254,77.641212,Nightclub
8,Domlur,12.9608,77.6361,Anand Sweets,12.960166,77.645168,Indian Restaurant
9,Domlur,12.9608,77.6361,Drops Total Spirits,12.964527,77.641636,Liquor Store


In [23]:
bengaluru_venues.shape

(736, 7)

### So, we can see total 736 venues are returned for Bengaluru.
### Now lets see how many venues are returned for each neighborhood of the city

In [25]:
bengaluru_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
BTM Layout,50,50,50,50,50,50
Bellandur,10,10,10,10,10,10
Bommanahalli,8,8,8,8,8,8
Bommasandra,4,4,4,4,4,4
Domlur,65,65,65,65,65,65
Electronic City,28,28,28,28,28,28
Hebbal,8,8,8,8,8,8
Hoodi,16,16,16,16,16,16
Indiranagar,97,97,97,97,97,97


## Let's find out how many unique categories can be curated from all the returned venues

In [26]:
print('There are {} uniques categories.'.format(len(bengaluru_venues['Venue Category'].unique())))

There are 127 uniques categories.


In [27]:
# one hot encoding
bengaluru_onehot = pd.get_dummies(bengaluru_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bengaluru_onehot['Neighborhood'] = bengaluru_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bengaluru_onehot.columns[-1]] + list(bengaluru_onehot.columns[:-1])
bengaluru_onehot = bengaluru_onehot[fixed_columns]


In [28]:
bengaluru_onehot.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Asian Restaurant,Auto Dealership,BBQ Joint,...,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Domlur,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Domlur,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Domlur,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Domlur,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Domlur,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Next, we will group rows by neighborhood and check the frequency of each category to find the top venues for each neighborhood

In [29]:
bengaluru_grouped = bengaluru_onehot.groupby('Neighborhood').mean().reset_index()

In [30]:
bengaluru_grouped.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Asian Restaurant,Auto Dealership,BBQ Joint,...,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,BTM Layout,0.0,0.0,0.0,0.0,0.0,0.000000,0.020000,0.0,0.020000,...,0.020000,0.0,0.0,0.0,0.0,0.08,0.000000,0.0,0.0,0.0
1,Bellandur,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0
2,Bommanahalli,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.125000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0
3,Bommasandra,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0
4,Domlur,0.0,0.0,0.0,0.0,0.0,0.015385,0.015385,0.0,0.015385,...,0.015385,0.0,0.0,0.0,0.0,0.00,0.015385,0.0,0.0,0.0


## Now we will find out top 10 venues of each neighborhood
### For that lets write a function to sort the frequency of the venues

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bengaluru_grouped['Neighborhood']

for ind in np.arange(bengaluru_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bengaluru_grouped.iloc[ind, :], num_top_venues)


In [35]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BTM Layout,Indian Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,Coffee Shop,Café,Bakery,Pizza Place,Snack Place,Chinese Restaurant,Park
1,Bellandur,Sports Bar,Badminton Court,Department Store,Indian Restaurant,Gym / Fitness Center,Other Great Outdoors,General Entertainment,Gastropub,Café,Farmers Market
2,Bommanahalli,Hotel Bar,Tea Room,Hotel Pool,Department Store,Cafeteria,Furniture / Home Store,Pizza Place,Indian Restaurant,Grocery Store,Farmers Market
3,Bommasandra,South Indian Restaurant,Sporting Goods Shop,Coffee Shop,Print Shop,Gym / Fitness Center,Gym,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
4,Domlur,Indian Restaurant,Café,Coffee Shop,Hotel,Italian Restaurant,Sports Bar,Food Truck,Burger Joint,Burrito Place,Pub


## Now we will cluster the neighborhoods

In [79]:
# set number of clusters
kclusters = 3

bengaluru_grouped_clustering = bengaluru_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bengaluru_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]                                

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

In [81]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [82]:
bengaluru_merged = df_blr
bengaluru_merged.rename(columns={'Name' : 'Neighborhood'},inplace=True)

In [83]:
# merge bengaluru_grouped with df_blr to add latitude/longitude for each neighborhood
bengaluru_merged = bengaluru_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [84]:
bengaluru_merged

,Zipcode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,560071,Domlur,12.9608,77.6361,0,Indian Restaurant,Café,Coffee Shop,Hotel,Italian Restaurant,Sports Bar,Food Truck,Burger Joint,Burrito Place,Pub
1,560038,Indiranagar,12.9699,77.6499,0,Indian Restaurant,Ice Cream Shop,Pub,Bakery,Lounge,Chinese Restaurant,Café,Burger Joint,Restaurant,Cocktail Bar
2,560003,Malleswaram,13.0031,77.5643,0,Indian Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,South Indian Restaurant,Coffee Shop,Café,Breakfast Spot,Light Rail Station,Park,Pizza Place
3,560001,Shivajinagar,12.9857,77.6057,0,Indian Restaurant,Clothing Store,Hotel,Fast Food Restaurant,Women's Store,Bar,Shopping Mall,Café,Candy Store,Men's Store
4,560103,Bellandur,12.9034,77.6784,0,Sports Bar,Badminton Court,Department Store,Indian Restaurant,Gym / Fitness Center,Other Great Outdoors,General Entertainment,Gastropub,Café,Farmers Market
5,560048,Hoodi,12.9896,77.7127,0,Indian Restaurant,Paper / Office Supplies Store,Supermarket,Salad Place,Bus Station,Cafeteria,Soccer Field,Bed & Breakfast,Gastropub,Hotel
6,560037,Marathalli,12.9569,77.7011,0,Indian Restaurant,Clothing Store,Vegetarian / Vegan Restaurant,Dessert Shop,Sports Bar,Department Store,Hotel,Gastropub,Andhra Restaurant,Chinese Restaurant
7,560066,Whitefield,12.9698,77.7500,0,Brewery,Café,Indian Restaurant,Bakery,Hotel Bar,Restaurant,Eastern European Restaurant,Kerala Restaurant,Grocery Store,Gym / Fitness Center
8,560043,Kammanahalli,13.0159,77.6379,0,Indian Restaurant,Fast Food Restaurant,Restaurant,BBQ Joint,Chinese Restaurant,Department Store,Bakery,Ice Cream Shop,Korean Restaurant,Café
9,560024,Hebbal,13.0354,77.5988,0,Indian Restaurant,Market,Pizza Place,Department Store,Park,Café,Eastern European Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


## Result : <br> We will label each cluster as follows

### Cluster 1 : "Indian Restaurant" <br> Cluster 2 : "Sports/Gym Center" <br>Cluster 3 : "Market"

## Lets create seperate table for each cluster with top 5 venues for each neighborhood.

In [103]:
bengaluru_c1=pd.DataFrame(bengaluru_merged.loc[bengaluru_merged['Cluster Labels'] == 0,bengaluru_merged.columns[[1] + list(range(5,10))]]).reset_index(drop=True)

In [104]:
bengaluru_c1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Domlur,Indian Restaurant,Café,Coffee Shop,Hotel,Italian Restaurant
1,Indiranagar,Indian Restaurant,Ice Cream Shop,Pub,Bakery,Lounge
2,Malleswaram,Indian Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,South Indian Restaurant,Coffee Shop
3,Shivajinagar,Indian Restaurant,Clothing Store,Hotel,Fast Food Restaurant,Women's Store
4,Bellandur,Sports Bar,Badminton Court,Department Store,Indian Restaurant,Gym / Fitness Center
5,Hoodi,Indian Restaurant,Paper / Office Supplies Store,Supermarket,Salad Place,Bus Station
6,Marathalli,Indian Restaurant,Clothing Store,Vegetarian / Vegan Restaurant,Dessert Shop,Sports Bar
7,Whitefield,Brewery,Café,Indian Restaurant,Bakery,Hotel Bar
8,Kammanahalli,Indian Restaurant,Fast Food Restaurant,Restaurant,BBQ Joint,Chinese Restaurant
9,Hebbal,Indian Restaurant,Market,Pizza Place,Department Store,Park


In [107]:
bengaluru_c2=pd.DataFrame(bengaluru_merged.loc[bengaluru_merged['Cluster Labels'] == 1,bengaluru_merged.columns[[1] + list(range(5,10))]]).reset_index(drop=True)
bengaluru_c2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bommasandra,South Indian Restaurant,Sporting Goods Shop,Coffee Shop,Print Shop,Gym / Fitness Center


In [106]:
bengaluru_c3=pd.DataFrame(bengaluru_merged.loc[bengaluru_merged['Cluster Labels'] == 2,bengaluru_merged.columns[[1] + list(range(5,10))]]).reset_index(drop=True)
bengaluru_c3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Yelahanka,ATM,Indian Restaurant,Restaurant,Farmers Market,Flea Market


# Conclusion

### Looking at the clusters a person can make decision for investing on type or place for business.
### For example if a person wants to invest on openening a restaurant then he can look on cluster 3 where there are very few restaurants.

# Lets visualize the cluster on Bengaluru Map

In [109]:
# create map
map_clusters = folium.Map(location=[latitude_blr, longitude_blr], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bengaluru_merged['Latitude'], bengaluru_merged['Longitude'], bengaluru_merged['Neighborhood'], bengaluru_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters